# Lab 1: Korean NLI (Natural Language Inference) Training on AWS

## Introduction
---

본 모듈에서는 허깅페이스 트랜스포머(Hugging Face transformers) 라이브러리를 사용하여 한국어 자연어 추론 (Korean NLI; Natural Language Inference) 쌍을 훈련합니다. 자연어 추론은 전제(premise)와 가설(hypothesis)이 포함된 두 문장 사이에서 전제가 참이라고 가정할 때, 가설의 연결이 참인지(entailment), 모순이 있는지(contradiction), 알 수 없는지(neutral)로 구별하는 다운스트림 태스크입니다.

***[Note] SageMaker Studio Lab, SageMaker Studio, SageMaker 노트북 인스턴스, 또는 여러분의 로컬 머신에서 이 데모를 실행할 수 있습니다. SageMaker Studio Lab을 사용하는 경우 GPU를 활성화하세요.***

### References
- Hugging Face Tutorial: https://huggingface.co/docs/transformers/tasks/question_answering
- Fine-tuning with custom datasets: https://huggingface.co/transformers/v4.11.3/custom_datasets.html#question-answering-with-squad-2-0
- KorNLI datasets: https://github.com/kakaobrain/KorNLUDatasets/tree/master/KorNLI
- KLUE: https://github.com/KLUE-benchmark/KLUE


## 1. Setup Environments
---

### Import modules

In [1]:
import os
import sys
import json
import logging
import argparse
import torch
from torch import nn
import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython.display import display, HTML

from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, set_seed
)
from transformers.trainer_utils import get_last_checkpoint
from datasets import load_dataset, load_metric, ClassLabel, Sequence

logging.basicConfig(
    level=logging.INFO, 
    format='[{%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

### Argument parser

In [2]:
def parser_args(train_notebook=False):
    parser = argparse.ArgumentParser()

    # Default Setting
    parser.add_argument("--epochs", type=int, default=3)
    parser.add_argument("--seed", type=int, default=42)
    parser.add_argument("--train_batch_size", type=int, default=32)
    parser.add_argument("--eval_batch_size", type=int, default=64)
    parser.add_argument("--max_length", type=int, default=384)
    parser.add_argument("--stride", type=int, default=64)
    parser.add_argument("--warmup_steps", type=int, default=100)
    parser.add_argument("--logging_steps", type=int, default=100)
    parser.add_argument("--learning_rate", type=str, default=3e-5)
    parser.add_argument("--disable_tqdm", type=bool, default=False)
    parser.add_argument("--fp16", type=bool, default=True)
    parser.add_argument("--tokenizer_id", type=str, default='klue/roberta-base')
    parser.add_argument("--model_id", type=str, default='klue/roberta-base')
    
    # SageMaker Container environment
    parser.add_argument("--output_data_dir", type=str, default=os.environ["SM_OUTPUT_DATA_DIR"])
    parser.add_argument("--model_dir", type=str, default=os.environ["SM_MODEL_DIR"])
    parser.add_argument("--n_gpus", type=str, default=os.environ["SM_NUM_GPUS"])
    parser.add_argument("--train_dir", type=str, default=os.environ["SM_CHANNEL_TRAIN"])
    parser.add_argument("--valid_dir", type=str, default=os.environ["SM_CHANNEL_VALID"])
    parser.add_argument('--chkpt_dir', type=str, default='/opt/ml/checkpoints')     

    if train_notebook:
        args = parser.parse_args([])
    else:
        args = parser.parse_args()
    return args

In [3]:
train_dir = 'nli_train'
valid_dir = 'nli_valid'
!rm -rf {train_dir} {valid_dir}
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True) 

### Load Arguments

주피터 노트북에서 곧바로 실행할 수 있도록 설정값들을 로드합니다. 물론 노트북 환경이 아닌 커맨드라인에서도 `cd scripts & python3 train.py` 커맨드로 훈련 스크립트를 실행할 수 있습니다.

In [4]:
chkpt_dir = 'chkpt'
model_dir = 'model'
output_data_dir = 'data'
num_gpus = torch.cuda.device_count()

!rm -rf {chkpt_dir} {model_dir} {output_data_dir} 

if os.environ.get('SM_CURRENT_HOST') is None:
    is_sm_container = False

    #src_dir = '/'.join(os.getcwd().split('/')[:-1])
    src_dir = os.getcwd()
    os.environ['SM_MODEL_DIR'] = f'{src_dir}/{model_dir}'
    os.environ['SM_OUTPUT_DATA_DIR'] = f'{src_dir}/{output_data_dir}'
    os.environ['SM_NUM_GPUS'] = str(num_gpus)
    os.environ['SM_CHANNEL_TRAIN'] = f'{src_dir}/{train_dir}'
    os.environ['SM_CHANNEL_VALID'] = f'{src_dir}/{valid_dir}'

args = parser_args(train_notebook=True) 
args.chkpt_dir = chkpt_dir
logger.info("***** Arguments *****")
logger.info(''.join(f'{k}={v}\n' for k, v in vars(args).items()))

os.makedirs(args.chkpt_dir, exist_ok=True) 
os.makedirs(args.model_dir, exist_ok=True)
os.makedirs(args.output_data_dir, exist_ok=True) 

[{204499775.py:21} INFO - ***** Arguments *****
[{204499775.py:22} INFO - epochs=3
seed=42
train_batch_size=32
eval_batch_size=64
max_length=384
stride=64
warmup_steps=100
logging_steps=100
learning_rate=3e-05
disable_tqdm=False
fp16=True
tokenizer_id=klue/roberta-base
model_id=klue/roberta-base
output_data_dir=/home/ec2-user/SageMaker/sm-kornlp-usecases/nli/data
model_dir=/home/ec2-user/SageMaker/sm-kornlp-usecases/nli/model
n_gpus=4
train_dir=/home/ec2-user/SageMaker/sm-kornlp-usecases/nli/nli_train
valid_dir=/home/ec2-user/SageMaker/sm-kornlp-usecases/nli/nli_valid
chkpt_dir=chkpt



<br>

## 2. Preparation & Custructing Feature set
---

### Dataset

본 핸즈온에서 사용할 데이터셋은 KLUE-NLI로 허깅페이스의 dataset 라이브러리로 곧바로 로드할 수 있습니다.
- KLUE: https://github.com/KLUE-benchmark/KLUE

In [5]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(
                lambda x: [typ.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))

In [6]:
datasets = load_dataset('klue', 'nli')
show_random_elements(datasets["train"])

[{builder.py:641} WARNING - Reusing dataset klue (/home/ec2-user/.cache/huggingface/datasets/klue/nli/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)


  0%|          | 0/2 [00:00<?, ?it/s]

,guid,source,premise,hypothesis,label
0,klue-nli-v1_train_03020,wikipedia,그는 1903년에 자신과 아내 파울리네 사이에 있었던 한 부부 싸움을 떠올렸다.,그는 1903년 다음 해에 파울리네와 부부가 되었다.,contradiction
1,klue-nli-v1_train_20075,airbnb,지금껏 다녔던 숙소보다 너무 좋았어요.,지금껏 다녔던 숙소 중에 제일 별로였어요.,contradiction
2,klue-nli-v1_train_19388,wikinews,"조선민주주의인민공화국이 2013년 2월 12일, 제3차 핵실험을 성공하였다고 공식 발표하였다.",조선민주주의인민공화국은 핵실험에 실패한 적이 있다.,neutral
3,klue-nli-v1_train_03544,wikinews,"그런데, 아직도 문제를 단순히 공식만으로 풀게 하고, 지루하게 계산만을 반복시키는 그런 수학, 이거 안 통합니다.",이제 더 이상 수학에 공식외우기는 없어야 합니다.,neutral
4,klue-nli-v1_train_03100,wikitree,"그는 주머니에서 오만 원권 두 장을 꺼내 팬에게 건넸고, 공연장에 있던 사람들은 그의 행동에 환호했다.",그는 주머니에서 만원 권 두 장만을 꺼내 팬에게 건넸다.,contradiction
5,klue-nli-v1_train_03446,policy,"그러나, 앞날을 결코 낙관할 수 없습니다.",낙관할 수 없는 앞날입니다.,entailment
6,klue-nli-v1_train_01274,NSMC,감독이라는것도 정말 일정 자격 시험이 필요한게 아닐까 하는 생각이 들게 한다,일정 자격 시험을 통과해서 감독 자격을 받도록 해야 하는게 아닐까 하는 생각이 들게 한다.,entailment
7,klue-nli-v1_train_22075,NSMC,타란티노가 연출했다면 더 잘 살렸을듯 연출이 각본을 따라가질못한다,타란티노가 연출했다면 더 좋았을 듯.,entailment
8,klue-nli-v1_train_09824,airbnb,빵이나 커피 잼 과일 맘껏 먹을 수 있고요,잼은 두 종류 뿐이고요.,neutral
9,klue-nli-v1_train_23753,wikitree,"한편, 구는 수립된 안전관리계획을 책자로 제작해 관련 유관기관 등에 배부할 예정이다.",안전관리계획을 책자로 제작하는 곳은 유관기관이다.,contradiction


### Tokenization


데이터셋을 토큰화합니다. 한 가지 눈여겨 보실 점이 `tokenizer()` 함수의 인자값에 `return_token_type_ids=False`를 지정한 점입니다.
본 핸즈온에서는 RoBERTa 모델로 파인 튜닝을 수행하는데, RoBERTa는 BERT와 달리 다음 문장 여부를 나타내는 Next Sentence Prediction (NSP) 를 수행하지 않으므로, `token_type_ids`를 사용하지 않습니다.

토큰화에 대한 자세한 내용은 https://huggingface.co/docs/datasets/process#processing-data-with-map 를 참조하세요.

In [7]:
tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_id, use_fast=True)

# tokenizer helper function
def tokenize(examples, sentence1_key='premise', sentence2_key='hypothesis'):
    return tokenizer(
        examples[sentence1_key],
        examples[sentence2_key],
        max_length=384,
        truncation=True,
        return_token_type_ids=False,
    )

encoded_datasets = datasets.map(tokenize, batched=True)

# set format for pytorch
encoded_datasets.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

[{arrow_dataset.py:2597} WARNING - Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/klue/nli/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e/cache-990f8277f7ffe9bd.arrow


  0%|          | 0/3 [00:00<?, ?ba/s]

### Save Training/Evaluation data

In [11]:
train_dir = 'datasets/train'
test_dir = 'datasets/test'
!rm -rf {train_dir} {test_dir}

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True) 

train_dataset = encoded_datasets['train']
valid_dataset = encoded_datasets['validation']

if not os.listdir(args.train_dir):
    train_dataset.save_to_disk(args.train_dir)
if not os.listdir(args.valid_dir):
    valid_dataset.save_to_disk(args.valid_dir)

# from datasets import load_from_disk
# train_dataset = load_from_disk(args.train_dir)
# valid_dataset = load_from_disk(args.valid_dir)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


<br>

## 3. Training
---

### Define Custom metric
특정 시점마다(예: epoch, steps) 검증 데이터셋으로 정밀도(precision), 재현율(recall), F1 스코어, 정확도(accuracy)를 등의 지표를 계산하기 위한 커스텀 함수를 정의합니다.

커스텀 함수의 첫번째 인자는 `EvalPrediction` 객체로, 예측값(`predictios`)과 정답값(`label_ids`)를 포함합니다. 자세한 내용은 아래 웹사이트를 참조하세요. 
https://huggingface.co/transformers/internal/trainer_utils.html#transformers.EvalPrediction

In [13]:
metric = load_metric("glue", "qnli")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(args.model_id, num_labels=3)

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier

### Training Preparation

In [15]:
training_args = TrainingArguments(
    output_dir=args.chkpt_dir,
    overwrite_output_dir=True if get_last_checkpoint(args.chkpt_dir) is not None else False,
    num_train_epochs=args.epochs,
    per_device_train_batch_size=args.train_batch_size,
    per_device_eval_batch_size=args.eval_batch_size,
    warmup_steps=args.warmup_steps,
    weight_decay=0.01,
    learning_rate=float(args.learning_rate),
    evaluation_strategy="epoch",`
    save_strategy="epoch",
    metric_for_best_model="accuracy",
)

In [16]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

### Training
훈련을 수행합니다. 딥러닝 기반 자연어 처리 모델 훈련에는 GPU가 필수이며, 본격적인 훈련을 위해서는 멀티 GPU 및 분산 훈련을 권장합니다. 만약 멀티 GPU가 장착되어 있다면 Trainer에서 총 배치 크기 = 배치 크기 x GPU 개수로 지정한 다음 데이터 병렬화를 자동으로 수행합니다.

In [17]:
%%time
# train model
if get_last_checkpoint(args.chkpt_dir) is not None:
    logger.info("***** Continue Training *****")
    last_checkpoint = get_last_checkpoint(args.chkpt_dir)
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: source, guid, premise, hypothesis. If source, guid, premise, hypothesis are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 24998
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 588
/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/nn/parallel/_

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.504815,0.818333
2,No log,0.436733,0.847000
3,0.440700,0.450579,0.852667


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: source, guid, premise, hypothesis. If source, guid, premise, hypothesis are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3000
  Batch size = 256
Saving model checkpoint to chkpt/checkpoint-196
Configuration saved in chkpt/checkpoint-196/config.json
Model weights saved in chkpt/checkpoint-196/pytorch_model.bin
tokenizer config file saved in chkpt/checkpoint-196/tokenizer_config.json
Special tokens file saved in chkpt/checkpoint-196/special_tokens_map.json
/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0

CPU times: user 4min 13s, sys: 45.1 s, total: 4min 58s
Wall time: 4min 12s


<br>

## 4. Evaluation
---

평가를 수행합니다.

In [18]:
outputs = trainer.predict(valid_dataset)
eval_results = outputs.metrics

# writes eval result to file which can be accessed later in s3 ouput
with open(os.path.join(args.output_data_dir, "eval_results.txt"), "w") as writer:
    print(f"***** Evaluation results at {args.output_data_dir} *****")
    for key, value in sorted(eval_results.items()):
        writer.write(f"{key} = {value}\n")
        logger.info(f"{key} = {value}\n")

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: source, guid, premise, hypothesis. If source, guid, premise, hypothesis are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3000
  Batch size = 256
/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


***** Evaluation results at /home/ec2-user/SageMaker/sm-kornlp-usecases/nli/data *****
[{4005816495.py:9} INFO - test_accuracy = 0.8526666666666667

[{4005816495.py:9} INFO - test_loss = 0.4505786597728729

[{4005816495.py:9} INFO - test_runtime = 2.825

[{4005816495.py:9} INFO - test_samples_per_second = 1061.937

[{4005816495.py:9} INFO - test_steps_per_second = 4.248



In [19]:
pred_logits = outputs.predictions
true = outputs.label_ids.ravel()
pred = pred_logits.argmax(-1).ravel()

In [20]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
unique_labels = train_dataset.features['label'].names
print(classification_report(true, pred, target_names=unique_labels))

               precision    recall  f1-score   support

   entailment       0.86      0.88      0.87      1000
      neutral       0.84      0.85      0.85      1000
contradiction       0.85      0.83      0.84      1000

     accuracy                           0.85      3000
    macro avg       0.85      0.85      0.85      3000
 weighted avg       0.85      0.85      0.85      3000



<br>

## 5. Prediction
---

여러분만의 샘플 문장을 만들어서 자유롭게 추론을 수행해 보세요.

In [65]:
idx = [i for i in range(len(train_dataset.features['label'].names))]
classes = train_dataset.features['label'].names

model.config.label2id = dict(zip(classes, idx))
model.config.id2label = dict(zip(idx, classes))

In [66]:
from transformers import pipeline
classifier = pipeline(
    task="text-classification",
    model=model, 
    tokenizer=tokenizer,
    top_k=1,
    device=0
)

In [71]:
example = f"머신러닝은 쉽다. {tokenizer.sep_token} 머신러닝은 어렵다."
classifier(sentences)

[{'label': 'contradiction', 'score': 0.9946433305740356}]